In [3]:
from xlrd import *
from random import *
import math
import numpy as np
from numpy import linalg as la

def carregar_matriz():
    # arquivo = input("Digite o nome do arquivo com extensão:")
    arquivo = 'planilha.xlsx'
    planilha = open_workbook(arquivo).sheet_by_index(0)
    matriz = []

    for linha in range(planilha.nrows):
        matriz.append([])
        for col in range(0, planilha.ncols -1):
            matriz[linha].append(float(planilha.cell_value(linha, col)))

    return matriz

def carregar_vetorb():
    arquivo = 'planilha.xlsx'
    planilha = open_workbook(arquivo).sheet_by_index(0)
    vetor_b = []

    for linha in range(0, planilha.nrows):
        vetor_b.append(planilha.cell_value(linha, planilha.nrows))

    return vetor_b

def print_matriz(m):
    for i in range(len(m)):
        for j in range(len(m[i])):
            print("{0:.4f}  ".format(m[i][j]), end="")

        print()
        
def print_vetorb(v):
    for i in range(len(v)):
        print("{0:.4f}  ".format(v[i]))

In [4]:
class Questao1:
    def __init__(self):
        self.matriz = carregar_matriz()
        self.vetor_b = carregar_vetorb()
        self.fator_cholesky = []
        self.matriz_l = []
        self.matriz_u = []

        print_matriz(self.matriz)
        print_vetorb(self.vetor_b)
        letra = input("a) Avaliar se é triangular superior, inferior, diagonal ou nenhuma das três.\n"
                      "b) Calcular o fator de Cholesky.\n"
                      "c) Construir matrizes L e U.\n"
                      "d) Avaliar critérios: linhas, colunas, Sassenfeld e norma.\n"
                      "e) Método de Jacobi.\n"
                      "f) Método de Gauss-Seidel.\n"
                      "g) Método SOR.\n"
                      "h) Eliminação de Gauss.\n"
                      "i) Decomposição SDV.\n"
                      "k) Fatoração QR.\n"
                      "l) Calcular número condição.\n"
                      "m) Gradiente conjugado.\n"
                      )
        if letra == 'a':
            self.identificar_matriz()
        elif letra == 'b':
            self.calcular_fator_cholesky()
        elif letra == 'c':
            self.lu()
        elif letra == 'd':
            self.avaliar_criterios()
        elif letra == 'e':
            self.jacobi()
        elif letra == 'f':
            self.gauss_seidel()
        elif letra == 'g':
            pass
        elif letra == 'h':
            self.elimn_gauss()
        elif letra == 'i':
            pass
        elif letra == 'k':
            pass
        elif letra == 'l':
            self.num_condicao()
        elif letra == 'm':
            pass

    def identificar_matriz(self):
        superior = self.is_triangular_superior()
        inferior = self.is_triangular_inferior()
        diagonal = self.is_diagonal()

        if superior:
            superior = "é triangular superior"
        else:
            superior = "não é triangular superior"

        if inferior:
            inferior = "é triangular inferior"
        else:
            inferior = "não é triangular inferior"

        if diagonal:
            diagonal = "é diagonal"
        else:
            diagonal = "não é diagonal"
        print("A matriz A {}, {}, {}.".format(superior, inferior, diagonal))

    def calcular_fator_cholesky(self):
        positiva_definida = self.is_positiva_definida()
        if positiva_definida:
            for i in range(len(self.matriz)):
                self.fator_cholesky.append([])
                for j in range(len(self.matriz[i])):
                    self.fator_cholesky[i].append(0)

            self.fator_cholesky[0][0] = self.matriz[0][0]**(1/2)  # R 1x1

            self.fator_cholesky[0][1:] = \
                [self.matriz[0][j]/self.fator_cholesky[0][0] for j in range(1, len(self.matriz))]  # R 1xj j = 1...n

            self.fator_cholesky[1][1] = (self.matriz[1][1]-(self.fator_cholesky[0][1]**2))**(1/2)  # R 2x2

            self.fator_cholesky[1][2:] = \
                [(self.matriz[1][j] - (self.fator_cholesky[0][1]*self.fator_cholesky[0][j])) / self.fator_cholesky[1][1]
                 for j in range(2, len(self.matriz))]  # R 2xj j = 3...n

            for i in range(2, len(self.matriz)):
                soma = 0
                for k in range(i):
                    soma += self.fator_cholesky[k][i]**2
                self.fator_cholesky[i][i] = (self.matriz[i][i] - soma)**(1/2)

            for i in range(len(self.matriz)):
                for j in range((i+1), len(self.matriz[i])):
                    soma = 0
                    for k in range(i):
                        soma += self.fator_cholesky[k][i]*self.fator_cholesky[k][j]
                    self.fator_cholesky[i][j] = (self.matriz[i][j] - soma)/self.fator_cholesky[i][i]
            print_matriz(self.fator_cholesky)
        else:
            print("Não é possível calcular o Fator de Cholesky")

    def lu(self):
        if self.verificar_det_submatrizes():

            matriz = np.array(self.matriz)
            matriz_l = self.matriz_l
            matriz_u = self.matriz_u

            for i in range(len(matriz)):
                matriz_l.append([])
                matriz_u.append([])
                for j in range(len(matriz[i])):
                    if i == j:
                        matriz_l[i].append(1)
                    else:
                        matriz_l[i].append(0)
                    matriz_u[i].append(0)

            for j in range(len(matriz)):  # u0j = a0j
                matriz_u[0][j] = matriz[0][j]

            for i in range(1, len(matriz)):  # li0 = ai0/u10
                matriz_l[i][0] = matriz[i][0]/matriz_u[0][0]

            for k in range(1, len(matriz)):  # ukj = akj - soamatorio[0-(k-1), m](lkm * uml) k = 1..n
                soma = 0
                for j in range(k, len(matriz)):
                    for m in range(k-1):
                        soma += matriz_l[k][m] * matriz_u[m][j]
                    matriz_u[k][j] = matriz[k][j] - soma

            for k in range(2, len(matriz)):  # lik = aik - soamatorio[0-(k-1), m](lim * umk) k = 2..n
                soma = 0
                for i in range(k+1, len(matriz)):
                    for m in range(k-1):
                        soma += matriz_l[i][m] * matriz_u[m][k]
                    matriz_l[i][k] = matriz_u[k][k]**(-1) * (matriz[i][k] - soma)

            # print(np.array(matriz_l))
            # print(np.array(matriz_u))
            self.matriz_l = matriz_l
            self.matriz_u = matriz_u

        else:
            print("Um dos determinantes das submatrizes é igual a zero, "
                  "portanto não é possível prosseguir e encontrar as matrizes L e U")

    def avaliar_criterios(self):
        linhas = self.criterio_linhas()
        colunas = self.criterio_colunas()
        sassenfeld = self.criterio_sassenfeld()
        normas = self.criterio_normas()

        if linhas:
            linhas = "satisfaz o critério das linhas"
        else:
            linhas = "não satisfaz o critério das linhas"

        if colunas:
            colunas = "satisfaz o critério das colunas"
        else:
            colunas = "não satisfaz o critério das colunas"

        if sassenfeld:
            sassenfeld = "satisfaz o critério de Sassenfeld, ou seja," \
                         " a convergência do método de Gauss-Seidel é garantida"
        else:
            sassenfeld = "não satisfaz o critério de Sassenfeld"

        if normas:
            normas = "satisfaz o critério das normas"
        else:
            normas = "não satisfaz o critério das normas"

        print("A matriz A {}, {}, {}, {}.".format(linhas, colunas, sassenfeld, normas))

    def jacobi(self):

        matriz = np.array(self.matriz)
        vetor_b = np.array(self.vetor_b)
        vetor_x = []  # vetor contendo os valores inciais de x
        diagonal = []  # diagonal é a matriz que contém os elementos da diagonal pincipal da matriz A
        m = []  # M contendo todos os elementos da matriz A, menos a diagonal principal
        erro = float(input("Digite o valor da tolerancia:"))

        for i in range(len(matriz)):
            vetor_x.append(float(input("Digite o valor inicial para x{}:".format(i))))

        for i in range(len(matriz)):
            diagonal.append([])
            m.append([])
            for j in range(len(matriz)):
                diagonal[i].append(0)
                m[i].append(0)

        for i in range(len(matriz)):
            for j in range(len(matriz)):
                if i == j:
                    diagonal[i][j] = matriz[i][j]
                else:
                    m[i][j] = matriz[i][j]

        d_inversa = la.inv(np.array(diagonal))  # Retorna a inversa da matriz diagonal
        vetor_x = np.array(vetor_x)

        # Começo das iterações
        itera = 0
        while True:

            vetor_x0 = vetor_x

            vetor_x = d_inversa * vetor_b - d_inversa * m * vetor_x
            itera += 1

            resultado = vetor_x - vetor_x0

            maior_x = 0

            for i in range(len(resultado)):
                for j in range(len(resultado[i])):
                    if resultado[i][j] < 0:
                        if maior_x < -resultado[i][j]:
                            maior_x = float(-resultado[i][j])
                    else:
                        if maior_x < resultado[i][j]:
                            maior_x = float(resultado[i][j])

            if maior_x < erro:
                print('Método parou na {} iteração e obteve como resultado \nX = {}\ne erro = {}'.format
                      (itera, vetor_x.tolist(), maior_x))
                break

    def gauss_seidel(self):

        matrix = self.matriz
        vetor_b = self.vetor_b

        print('Método de Gauss-Seidel')

        tolerancia = float(input("Qual eh a tolerancia? "))

        x = np.zeros(len(matrix))
        k = 0

        solucao_ant = np.zeros(len(matrix))

        for i in solucao_ant:
            i = math.inf

        flag = 0

        while True:

            suma = 0
            k = k + 1
            for r in range(0, len(matrix)):
                suma = 0
                for c in range(0, len(matrix[r])-1):
                    if c != r:
                        suma = suma + matrix[r][c] * x[c]
                x[r] = (vetor_b[r] - suma) / matrix[r][r]
                print("x[" + str(r) + "]: " + str(x[r]))

            for j in range(0, len(x)):

                if abs(x[j] - solucao_ant[j]) < tolerancia:
                    flag = 1

                else:
                    flag = 0
            if flag == 1:
                break

            solucao_ant = x.copy()

    def sor(self):
        pass

    def elimn_gauss(self):

        matrix = self.matriz
        vetor_b = self.vetor_b

        m = len(matrix)
        n = len(matrix[0])

        x = np.zeros(m)

        for k in range(0, m):
            for r in range(k + 1, m):
                factor = (matrix[r][k] / matrix[k][k])
                vetor_b[r] = vetor_b[r] - (factor * vetor_b[k])
                for c in range(0, n):
                    matrix[r][c] = matrix[r][c] - (factor * matrix[k][c])

        # substituição pra trás

        x[m - 1] = vetor_b[m - 1] / matrix[m - 1][m - 1]

        print(x[m - 1])

        for r in range(m - 2, -1, -1):
            soma = 0
            for c in range(0, n):
                soma = soma + matrix[r][c] * x[c]
            x[r] = (vetor_b[r] - soma) / matrix[r][r]

        print("Resultado da matriz: ")

        for linha in matrix:
            for elemento in linha:
                print(elemento, end=" ")
            print()

        print("Resultado do vetor b: ")

        for elemento in vetor_b:
            print(elemento)

        print("Resultados: ")

        print(x)

    def decom_svd(self):
        pass

    def qr(self):
        pass

    def num_condicao(self):

        from scipy import linalg

        arquivo = 'planilha.xlsx'
        planilha = open_workbook(arquivo).sheet_by_index(0)

        m = planilha.nrows
        n = planilha.ncols

        matrix = self.matriz

        A_inversa = linalg.inv(matrix)
        print('Matriz A inversa:')
        for i in range(0, m):
            for j in range(0, n-1):
                sys.stdout.write('%.2f\t' % A_inversa[i][j])
            print()
        print()

        # Número condição de A: fazer norma infinita de linha
        # Norma infinita de linha da matriz A
        maxA = 0
        for i in range(0, m, 1):
            total = 0
            for j in range(0, n-1, 1):
                total += math.fabs(matrix[i][j])
            if maxA < total:
                maxA = total

        # Norma infinita de linha da matriz A inversa
        maxAI = 0
        for i in range(0, m, 1):
            total = 0
            for j in range(0, n-1, 1):
                total += math.fabs(A_inversa[i][j])
            if maxAI < total:
                maxAI = total

        # Cálculo do número condição
        num_cond = maxA * maxAI
        print('Número condição:', num_cond)
        print()

    def gradiente_conjugado(self):
        pass

    def is_diagonal(self):
        for i in range(len(self.matriz)):
            for j in range(len(self.matriz[i])):
                if i != j and self.matriz[i][j] != 0:
                    return False
        return True

    def is_triangular_superior(self):
        for i in range(len(self.matriz)):
            for j in range(len(self.matriz[i])):
                if i > j and self.matriz[i][j] != 0:
                    return False
        return True

    def is_triangular_inferior(self):
        for i in range(len(self.matriz)):
            for j in range(len(self.matriz[i])):
                if i < j and self.matriz[i][j] != 0:
                    return False
        return True

    def is_simetrica(self):
        for i in range(len(self.matriz)):
            for j in range(len(self.matriz[i])):
                if i > j and self.matriz[i][j] != self.matriz[j][i]:
                    return False
        return True

    def is_positiva_definida(self):
        if len(self.matriz) == len(self.matriz[0]):
            nn = True
        else:
            nn = False
        simetrica = self.is_simetrica()

        vet_x = []
        for i in range(len(self.matriz)):
            vet_x.append(randint(1, 1000))

        vet_matriz = []
        for i in range(len(self.matriz)):
            valor = 0
            for j in range(len(self.matriz[i])):
                valor += vet_x[j] * self.matriz[j][i]
            vet_matriz.append(valor)

        valor = 0
        for i in range(len(vet_matriz)):
            valor += vet_matriz[i] * vet_x[i]

        if nn and simetrica and valor > 0:
            return True
        else:
            if not nn:
                print("A matriz A não possui número igual de linhas x colunas:\nL: {} X C: {}.\n"
                      .format(len(self.matriz), len(self.matriz[0])))
            if not simetrica:
                print("A matriz não é simétrica.")
            if valor <= 0:
                print("xT*A*x <= 0.")
            return False

    def criterio_linhas(self):
        matriz = self.matriz

        for i in range(len(matriz)):
            soma = 0
            for j in range(len(matriz[i])):
                if i != j:
                    soma += matriz[i][j]
            if abs(soma) >= abs(matriz[i][i]):
                return False

        return True

    def criterio_colunas(self):
        matriz = self.matriz

        for j in range(len(matriz[0])):
            soma = 0
            for i in range(len(matriz)):
                if i != j:
                    soma += matriz[i][j]
            if abs(soma) >= abs(matriz[j][j]):
                return False

        return True

    def criterio_sassenfeld(self):
        matriz = self.matriz
        b = []

        for j in range(len(matriz[0])):
            soma = 0
            for i in range(len(matriz)):
                if i != j:
                    soma += matriz[i][j]  # Ainda necessario ver como fica o somatório aqui
            b.append(soma / abs(matriz[j][j]))

        if max(b) < 1:
            return True
        else:
            return False

    def criterio_normas(self):
        return False

    def verificar_det_submatrizes(self):
        matriz = self.matriz
        d = 0

        for u in range(len(matriz)):
            sub_matriz = []

            for i in range(len(matriz) - d):
                if d > 0:
                    sub_matriz.append(matriz[i][:-d])
                else:
                    sub_matriz.append(matriz[i][:])

            if la.det(sub_matriz) == 0:
                return False
            d += 1

        return True


In [8]:
class Questao4:
    def __init__(self):
        letra = input("a) Calcular Newton Rhapson para um polinômio de grau 10.\n"
                      "b) Comparar raizes analíticas com Newton Rhapson.\n"
                      )
        if letra == 'a':
            self.calcular_newton_rapshon()
        elif letra == 'b':
            self.calcular_raizes()

    def calcular_newton_rapshon(self):

        coeficientes = []  # vetor contendo os coeficientes
        for i in range(11)[::-1]:
            coef = float(input("Digite o coeficiente para o x^{}:".format(i)))
            coeficientes.append(coef)
        # print("A função ficou com a seguinte cara:\n"
        #       "{}x^10 + {}x^9 + {}x^8 + {}x^7 + {}x^6 + {}x^5 + {}x^4 + {}x^3 + {}x^2 + {}x + {}".format
        #       (coeficientes[0], coeficientes[1], coeficientes[2], coeficientes[3], coeficientes[4],
        #        coeficientes[5], coeficientes[6], coeficientes[7], coeficientes[8], coeficientes[9], coeficientes[10]))

        x0 = float(input("Digite o valor do x incial:"))  # substituir pelo velor do x inicial
        e = float(input("Digite o valor da tolerancia:"))

        i = 0
        x = [x0]
        er = [e + 1.0, e + 1.0]

        while er[0] > e and er[1] > e:
            p1 = self.func(x[i], coeficientes) / self.der(x[i], coeficientes)
            p2 = x[i] - p1

            x.append(p2)
            print(x[i])

            er = self.E(x[i], x[i + 1], self.func)

            i += 1

        return x

    def calcular_raizes(self):

        coeficientes = []  # vetor contendo os coeficientes
        for i in range(4)[::-1]:
            coef = float(input("Digite o coeficiente para o x^{}:".format(i)))
            coeficientes.append(coef)

        possiveis_raizes = []
        raizes_fracionarias = []
        term_indep = int(coeficientes[-1])
        for raiz in range(1, abs(term_indep) + 1):
            if (abs(term_indep) % raiz) == 0:
                possiveis_raizes.append(-raiz)
                possiveis_raizes.append(raiz)
                if raiz / coeficientes[0] not in possiveis_raizes:
                    raizes_fracionarias.append((-raiz) / coeficientes[0])
                    raizes_fracionarias.append(raiz / coeficientes[0])
        possiveis_raizes.extend(raizes_fracionarias)
        # print(possiveis_raizes)

        raizes = []
        for raiz in possiveis_raizes:
            if self.func(raiz, coeficientes) == 0:
                raizes.append(raiz)
                break
        if raizes:
            
            # Briot Ruffini
            termo = coeficientes[0] * raizes[0]
            coef_d = [termo]
            # print("{} = {} * {}".format(termo, raizes[0], termo / raizes[0]))
            for i in range(1, len(coeficientes) - 1):
                termo += coeficientes[i]
                # print("{} = {} + {}".format(termo, termo - coeficientes[i], coeficientes[i]))
                termo *= raizes[0]
                # print("{} = {} * {}".format(termo, raizes[0], termo / raizes[0]))
                coef_d.append(termo)
    
            termo += coeficientes[-1]
            # print("{} = {} + {}".format(termo, termo - coeficientes[-1], coeficientes[-1]))
    
            # Baskara
            raizes.append((-coef_d[1] - ((coef_d[1] ** 2) - (4 * coef_d[0] * coef_d[2])) ** (1 / 2)) / (2 * coef_d[0]))
            raizes.append((-coef_d[1] + ((coef_d[1] ** 2) - (4 * coef_d[0] * coef_d[2])) ** (1 / 2)) / (2 * coef_d[0]))
            print(raizes)
        else:
            print("Não encontrada solução analítica.")

        x0 = float(input("Digite o valor do x incial:"))  # substituir pelo velor do x inicial
        e = float(input("Digite o valor da tolerancia:"))

        i = 0
        x = [x0]
        er = [e + 1.0, e + 1.0]

        while er[0] > e and er[1] > e:
            p1 = self.func(x[i], coeficientes) / self.der(x[i], coeficientes)
            p2 = x[i] - p1

            x.append(p2)
            print(x[i])

            er = self.E(x[i], x[i + 1], self.func)

            i += 1

        print("Raiz encontrada por Newton-Rhapson:", x[-1])
        
    def func(self, x, coeficientes):
            y = 0
            ex = len(coeficientes)-1
            for c in coeficientes:
                y += c * (x ** ex)
                ex -= 1

            return y

    def der(self, x, coeficientes):
        y = 0
        ex = len(coeficientes)-1
        for c in coeficientes:
            if (ex - 1) >= 0:
                y += (ex * c) * (x ** (ex - 1))
            ex -= 1

        return y

    def E(self, x0, x1, func):
        return [abs(func(x1)), abs(x1 - x0)]
